In [23]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow import keras

In [14]:
# Load CSV file (exported from InfluxDB)
df = pd.read_csv("./data/battery_data.csv")

# Convert timestamp to datetime for time-series processing
df["timestamp"] = pd.to_datetime(df["timestamp"])

# Simulate Time-to-Failure (assuming failure happens at the last recorded timestamp)
df["timeBeforeFailure"] = (df["timestamp"].max() - df["timestamp"]).dt.total_seconds() / 3600  # Convert to hours

# Display processed data
print(df.head())

                         timestamp  batteryId  ambientTemp  batteryCurrent  \
0 2025-02-12 14:26:34.190000+00:00          1        18.65           78.06   
1 2025-02-12 14:26:37.025000+00:00          1        18.36           81.42   
2 2025-02-12 14:26:40.020000+00:00          1        18.17           75.76   
3 2025-02-12 14:26:43.024000+00:00          1        18.50           72.11   
4 2025-02-12 14:26:46.023000+00:00          1        18.12           73.72   

   batteryTemp  batteryVoltage  distance     kmh  stateOfCharge  \
0        25.22          396.39      0.20  127.75         0.9991   
1        25.33          396.23      0.31  130.46         0.9986   
2        25.39          396.49      0.42  125.59         0.9981   
3        25.44          396.66      0.52  122.42         0.9977   
4        25.47          396.58      0.62  123.87         0.9972   

   stateOfHealth  timeBeforeFailure  
0        99.9998           0.425357  
1        99.9997           0.424569  
2        99.99

In [15]:
# Select Features & Target Variable
features = ["batteryTemp", "batteryCurrent", "batteryVoltage", "stateOfCharge", "stateOfHealth"]
target = "timeBeforeFailure"

X = df[features]
y = df[target]

In [10]:
# Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalize data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [19]:
# Define neural network
model = keras.Sequential([
    keras.layers.Dense(128, activation="relu", input_shape=(X_train.shape[1],)),
    keras.layers.Dense(64, activation="relu"),
    keras.layers.Dense(32, activation="relu"),
    keras.layers.Dense(1) 
])

# Compile model
model.compile(optimizer="adam", loss="mse", metrics=["mae"])

In [20]:
# Train model
model.fit(X_train_scaled, y_train, epochs=50, batch_size=32, validation_data=(X_test_scaled, y_test))

Epoch 1/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 2s 16ms/step - loss: 0.0165 - mae: 0.0972 - val_loss: 0.0098 - val_mae: 0.0704
Epoch 2/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0066 - mae: 0.0542 - val_loss: 0.0086 - val_mae: 0.0458
Epoch 3/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0048 - mae: 0.0384 - val_loss: 0.0078 - val_mae: 0.0502
Epoch 4/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0047 - mae: 0.0364 - val_loss: 0.0080 - val_mae: 0.0415
Epoch 5/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0050 - mae: 0.0336 - val_loss: 0.0092 - val_mae: 0.0460
Epoch 6/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0034 - mae: 0.0255 - val_loss: 0.0053 - val_mae: 0.0370
Epoch 7/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0033 - mae: 0.0294 - val_loss: 0.0055 - val_mae: 0.0348
Epoch 8/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0031 - mae: 0.0214 - val_loss: 0.0050 - val_mae: 0.0344
Epoch 9/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0035 - mae:

In [21]:
# Make Predictions
y_pred = model.predict(X_test_scaled).flatten()

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step


In [24]:
# Evaluate Model
mae = mean_absolute_error(y_test, y_pred)
print(f"Model MAE: {mae:.2f} hours before failure")

Model MAE: 0.04 hours before failure


In [25]:
# Save model as keras and scaler for inference
model.save("models/battery_ttf_model.keras")
print("Model saved as TensorFlow format in 'models/battery_ttf_model_tf.keras'")

Model saved as TensorFlow format in 'models/battery_ttf_model_tf.keras'


In [26]:
# Load model
model = tf.keras.models.load_model("models/battery_ttf_model.keras")

# Export SavedModel
tf.saved_model.save(model, "models/battery_ttf_model")

print("Model converted to SavedModel format")

INFO:tensorflow:Assets written to: models/battery_ttf_model/assets


INFO:tensorflow:Assets written to: models/battery_ttf_model/assets


Model converted to SavedModel format


In [27]:
!mo --saved_model_dir models/battery_ttf_model --output_dir models/battery_ttf_model

[ INFO ] MO command line tool is considered as the legacy conversion API as of OpenVINO 2023.2 release.
In 2025.0 MO command line tool and openvino.tools.mo.convert_model() will be removed. Please use OpenVINO Model Converter (OVC) or openvino.convert_model(). OVC represents a lightweight alternative of MO and provides simplified model conversion API. 
Find more information about transition from MO to OVC at https://docs.openvino.ai/2023.2/openvino_docs_OV_Converter_UG_prepare_model_convert_model_MO_OVC_transition.html
2025-03-14 12:52:45.748319: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-03-14 12:52:45.765885: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-03-14 12:52:45.770460: E external/local_xla/xla/s